<a href="https://colab.research.google.com/github/robssoares/HTML5_CSS/blob/master/Curso_Extensao_(2021)_Exemplo_2_Pr%C3%A9_processamento_de_Textos_Informa%C3%A7%C3%A3o_Geogr%C3%A1fica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Pré-processamento de Textos - Informação Geográfica

- Praticar conceitos básicos/fundamentais de extração de informação geográfica
- Entidades nomeadas e Geocoding
- Distância entre coordenadas geográficas
- Incorporando distância geográfica na similaridade textual
- Visualizando a rede 
- Visualizando em mapas e mapa de calor

# Importando bibliotecas

In [1]:
!pip install geocoder
import geocoder
!pip install haversine
from haversine import haversine, Unit
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor
from polyglot.downloader import downloader
from polyglot.text import Text
!polyglot download ner2.pt
!polyglot download ner2.en
import pandas as pd
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import cosine
import numpy as np
import networkx as nx
!pip install plotly.express
from plotly import graph_objs as go
import folium
!pip install gdown
!pip install git+https://github.com/rmarcacini/sentence-transformers


     |████████████████████████████████| 98 kB 3.8 MB/s 
     |████████████████████████████████| 126 kB 5.4 MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52578 sha256=6cb2f2670865d51a9b69cba712e7b967c69638c174b38da99d95b19f0ee228fe
  Stored in directory: /root/.cache/pip/wheels/09/bc/67/75c9de8e9726460bc0b101ad225ad025cb8ce9e0759beb9d52
Successfully built polyglot
     |████████████████████████████████| 299 kB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pyicu: filename=PyICU-2.8-cp37-cp37m-linux_x86_64.whl size=1377067 sha256=44380722374a21eecf87a2ce8f069cc102470c05f5b106dab7a6362b766b4933
  Stored in directory: /root/.cache/pip/wheels/14/bd/45/aeddc643bd0637c14fa27bffaee5b411cdc323f8bec76ad15e
Successfully built pyicu
     |████████████████████████████████| 41.4 MB 1.7 MB/s 
  Created wheel for pycld2: filename=pycld2-0.41-cp37-cp37m-l

# Exemplo de Extração de Entidades Nomeadas - Informação Geográfica

In [2]:
!polyglot download embeddings2.pt

[polyglot_data] Downloading package embeddings2.pt to
[polyglot_data]     /root/polyglot_data...


In [3]:
# identificando entidades nomeadas em textos
text = Text("A escola avançada de big data na USP de São Carlos - SP está ocorrendo hoje")
text.entities



[I-ORG(['USP']), I-LOC(['São', 'Carlos']), I-LOC(['SP'])]

In [4]:
# extraindo entidades geográficas (tipo I-LOC)
text = Text("A escola avançada de big data na USP de São Carlos - SP está ocorrendo hoje")
loc = ""
for ent in text.entities:
  if ent.tag=='I-LOC':
    for s in ent:
      loc += s+" "
print(loc)

São Carlos SP 


# Etapas de Geocoding - Extraindo Lat/Lng de uma entidade geográfica

In [5]:
# extraindo latitude / longitude
print("Geocoding loc: "+loc)
g = geocoder.geonames(loc, maxRows=1, key='3eabda_1')
for result in g:
  print("Country: "+result.country)
  print("State: "+result.state)
  print("Address: "+result.address)
  print("Lat: "+result.lat)
  print("Lng: "+result.lng)

Geocoding loc: São Carlos SP 
Country: Brazil
State: São Paulo
Address: São Carlos
Lat: -22.0175
Lng: -47.89083


# Calculando distância (haversine) entre coordenadas geográficas

In [7]:
# calculando distancia haversine entre duas localidades
loc1 = "São Carlos, SP"
loc2 = "São Paulo, SP"
g1 = geocoder.geonames(loc1, maxRows=1, key='3eabda_1')
g2 = geocoder.geonames(loc2, maxRows=1, key='3eabda_1')
dist_km = haversine( (float(g1[0].lat),float(g1[0].lng)), (float(g2[0].lat),float(g2[0].lng)))
print("Dist (km) = "+str(dist_km))

Dist (km) = 213.2808236620993


# Obtendo um dataset de exemplo
- altere a query para obter uma amostra diferente
- é uma base de ~2 milhões de eventos coletados em 2017

In [8]:
import urllib.parse
import pandas as pd
import requests
from io import StringIO

# obtendo dataset com uma amostra eventos

query = "febre amarela" # query para consultar na base de eventos (max. de 300 respostas)
query = urllib.parse.quote(query)

url = 'https://websensors.net.br/minicurso/2019/eventos-br-2017.php?q='+query
headers = {"User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.14; rv:66.0) Gecko/20100101 Firefox/66.0"}
req = requests.get(url, headers=headers)

pd.set_option('display.max_colwidth', None)
dataset = pd.read_csv(StringIO(req.text),sep='\t', lineterminator='\n')

dataset[['title','local','lat','lng']]

,title,local,lat,lng
0,"Após 2ª morte por febre amarela em SP, vacinação é intensificada - Globo.com",Ribeirao Preto (SP),-21.17,-47.81
1,Casos de febre amarela em macacos na região preocupam ... - Globo.com,Ribeirao Preto (SP),-21.17,-47.81
2,G1 - Casos de febre amarela em macacos na região preocupam ... - Globo.com,Sao Jose do Rio Preto (SP),-20.82,-49.37
3,"Após 2ª morte por febre amarela em SP, vacinação é intensificada - Globo.com",Sao Jose do Rio Preto (SP),-20.82,-49.37
4,Mortes por febre amarela põem em alerta cidades do norte paulista - Istoe,Sao Jose do Rio Preto (SP),-20.82,-49.37
...,...,...,...,...
295,Casos de febre amarela dobram na cidade mais afetada por surto ... - Jornal O Globo,Teofilo Otoni (MG),-17.85,-41.50
296,"Mortes suspeitas por febre amarela sobem para 47 em Minas, diz governo - Globo.com",Guarapari (ES),-20.65,-40.51
297,Áreas endêmicas de Febre Amarela são monitoradas no Maranhão,Sao Luis (MA),-2.53,-44.30
298,"Casos suspeitos de febre amarela em MG sobem para 152, com 47 mortes - Jornal O Globo",Coronel Fabriciano (MG),-19.51,-42.62


# Pré-processando os textos

In [12]:
!gdown https://drive.google.com/uc?id=1NV5t1YhyyOzMF5zAovfbSLdZZLvqrfZ_
!unzip distiluse-base-multilingual-cased.zip -d language_model
from sentence_transformers import SentenceTransformer, LoggingHandler
import numpy as np
import logging

#np.set_printoptions(threshold=100)
#logging.basicConfig(format='%(asctime)s - %(message)s',datefmt='%Y-%m-%d %H:%M:%S',level=logging.INFO,handlers=[LoggingHandler()])

language_model = SentenceTransformer('./language_model')

Downloading...
From: https://drive.google.com/uc?id=1NV5t1YhyyOzMF5zAovfbSLdZZLvqrfZ_
To: /content/distiluse-base-multilingual-cased.zip
100% 504M/504M [00:02<00:00, 171MB/s]
Archive:  distiluse-base-multilingual-cased.zip
replace language_model/0_DistilBERT/added_tokens.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: language_model/0_DistilBERT/added_tokens.json  
replace language_model/0_DistilBERT/config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: language_model/0_DistilBERT/config.json  
replace language_model/0_DistilBERT/pytorch_model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: language_model/0_DistilBERT/pytorch_model.bin  A

replace language_model/0_DistilBERT/sentence_distilbert_config.json? [y]es, [n]o, [A]ll, [N]one, [r]ename:   inflating: language_model/0_DistilBERT/sentence_distilbert_config.json  
  inflating: language_model/0_DistilBERT/special_tokens_map.json  
  inflating: language_model/0_DistilBERT/tokenizer_config.json  
  inf

In [14]:
dataset['dbert_vectors'] = list(language_model.encode(dataset['title'].to_list()))

# Gerando uma Rede que considera Proximidade de Conteúdo + Proximidade Geográfica

In [15]:
# calculando a rede de eventos por proximidade de conteudo+geografico

# computando dissimilaridade de cosseno

def dis_cosine(vector1, vector2):
  dcos = cosine(vector1, vector2)
  return dcos



def compute_network(dataset,seed=0,min_dcos=0.7,max_neighbors=3,min_dgeo=-1,max_nodes=1000):
  
  G=nx.Graph()
  visited = []
  visited.append(seed)
  total = len(dataset)
  temp = {}
  
  while(True):
  
    seed = visited.pop()
    temp[seed]=1
  
    neighbors = {}
    
    lat1 = dataset.iloc[seed,:]['lat']
    lng1 = dataset.iloc[seed,:]['lng']

    for i in range(0,total):
      if seed==i: continue
      if i in temp: continue

      if seed==i: continue
      if i in temp: continue
      dcos = dis_cosine(dataset.iloc[seed].dbert_vectors, dataset.iloc[i].dbert_vectors)
      cos = 1.0 - dcos

      lat2 = dataset.iloc[i,:]['lat']
      lng2 = dataset.iloc[i,:]['lng']
      
      if(min_dgeo != -1):
        dist_km = haversine( (lat1,lng1), (lat2,lng2) )
        if (dist_km >= min_dgeo): cos = cos/4 # penalty
      
      
      dcos = 1.0 - cos
      if dcos <= min_dcos:
        neighbors[i] = dcos

    sorted_x = sorted(neighbors.items(), key=lambda kv: kv[1])

    counter=0
    for item in sorted_x:
      G.add_edge(seed, item[0], weight=(1-item[1]))
      if item[0] not in temp: visited.append(item[0])
      counter+=1
      if (counter >= max_neighbors): break     

  
    if(len(G) >= max_nodes): break
    if(len(visited)==0): break
    
    
    
  return G
    
G = compute_network(dataset,min_dcos=0.80,min_dgeo=300,seed=288)
nx.info(G)

'Graph with 287 nodes and 810 edges'

# Visualizando a Rede (Proximidade de Conteúdo + Proximidade Geográfica)

In [16]:
# visualizando a rede por proximidade de conteudo+geografico
def plot_event_network():
  
  # plotando rede
  pos = nx.drawing.layout.spring_layout(G)
  for item in pos:
    G.nodes[item]['pos']=[pos[item][0],pos[item][1]]
    
  edge_x = []
  edge_y = []
  for edge in G.edges():
      x0, y0 = G.nodes[edge[0]]['pos']
      x1, y1 = G.nodes[edge[1]]['pos']
      edge_x.append(x0)
      edge_x.append(x1)
      edge_x.append(None)
      edge_y.append(y0)
      edge_y.append(y1)
      edge_y.append(None)

  edge_trace = go.Scatter(
      x=edge_x, y=edge_y,
      line=dict(width=0.5, color='#888'),
      hoverinfo='none',
      mode='lines')

  node_x = []
  node_y = []
  for node in G.nodes():
      x, y = G.nodes[node]['pos']
      node_x.append(x)
      node_y.append(y)

  node_trace = go.Scatter(
      x=node_x, y=node_y,
      mode='markers',
      hoverinfo='text',
      marker=dict(
          showscale=True,
          # colorscale options
          #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
          #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
          #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
          colorscale='YlGnBu',
          reversescale=True,
          color=[],
          size=10,
          colorbar=dict(
              thickness=15,
              title='Node Connections',
              xanchor='left',
              titleside='right'
          )))
  
  node_adjacencies = []
  node_text = []
  for node, adjacencies in enumerate(G.adjacency()):
      node_adjacencies.append(len(adjacencies[1]))
      
  for node in G.nodes():
    node_text.append(str(dataset.iloc[node,:]['title'])+" - "+str(dataset.iloc[node,:]['local']))

  node_trace.marker.color = node_adjacencies
  node_trace.text = node_text
  
  fig = go.Figure(data=[edge_trace, node_trace],
               layout=go.Layout(
                  title='Event Network',
                  showlegend=False,
                  hovermode='closest',
                  margin=dict(b=20,l=5,r=5,t=40),
                  annotations=[ dict(
                      text="",
                      showarrow=False,
                      xref="paper", yref="paper",
                      x=0.005, y=-0.002 ) ],
                  xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                  yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                  )
  fig.show()

plot_event_network()

# Visualizando no mapa (como ponteiros)

In [ ]:
# plotando eventos no mapa
m = folium.Map([-14.5931291,-56.6985808], zoom_start=4)

for node in G.nodes():
  lat = dataset.iloc[node,:]['lat']
  lng = dataset.iloc[node,:]['lng']
  text = dataset.iloc[node,:]['title']
  folium.Marker([lat, lng], popup=text).add_to(m)

m

# Visualizando como mapa de calor

In [ ]:
# plotando eventos no mapa com heatmap
from folium.plugins import HeatMap

m = folium.Map([-14.5931291,-56.6985808], zoom_start=4)
data_heatmap = []
for node in G.nodes():
  lat = dataset.iloc[node,:]['lat']
  lng = dataset.iloc[node,:]['lng']
  data_heatmap.append([lat,lng,1])

HeatMap(data_heatmap).add_to(m)

m